Merry Christmas everyone! 

[Advent of Code day 8](http://adventofcode.com/2018/day/8)

Ah, damn it... got a cfg. Oh for prolog...

Having complained a lot [yesterday](Advent of Code 2018- Day 07.ipynb) about inplace alterations to data structures in python, I'd completely forgotten about `pyrsistent`, so I'll use that today.

In [68]:
from pyrsistent import pvector

## Part 1

In [69]:
test_input="2 3 0 3 10 11 12 1 1 0 1 99 2 1 1 2"

input_v=pvector([int(x) for x in test_input.strip().split()])
input_v


pvector([2, 3, 0, 3, 10, 11, 12, 1, 1, 0, 1, 99, 2, 1, 1, 2])

Hmmm, that's interesting. There seems to be a little redundancy here. Or I've misinterpreted. Either way...

Next cell is the basic reducer (without accumulating the metadata)

In [70]:
def reduce(input_v):
    '''
    parse the node at the front of the input vector. Return pair
    of the metadata and the rest of the input.
    '''
    if input_v[0]==0:
        return(input_v[2:input_v[1]+2], input_v[input_v[1]+2:])
    else:
        to_parse_v=input_v[2:]
        for i in range(input_v[0]):
            (metadata_v, to_parse_v)=reduce(to_parse_v)
        return (to_parse_v[:input_v[1]], to_parse_v[input_v[1]:])

            



OK, now adjust `reduce` so that it accumulates the metadata

In [71]:
def reduce(input_v, metadata_sofar_v):
    '''
    parse the node at the front of the input vector. Return pair
    of all metadata collected and the rest of the input.
    '''
    if input_v[0]==0:
        return(metadata_sofar_v.extend(input_v[2:input_v[1]+2]),
               input_v[input_v[1]+2:])
    else:
        to_parse_v=input_v[2:]
        metadata_v=metadata_sofar_v
        for i in range(input_v[0]):
            (metadata_v, to_parse_v)=reduce(to_parse_v, metadata_v)
        return (metadata_v.extend(to_parse_v[:input_v[1]]), 
                to_parse_v[input_v[1]:])

In [72]:
test_input="2 3 0 3 10 11 12 1 1 0 1 99 2 1 1 2"

input_v=pvector([int(x) for x in test_input.strip().split()])
input_v


pvector([2, 3, 0, 3, 10, 11, 12, 1, 1, 0, 1, 99, 2, 1, 1, 2])

In [73]:
assert sum(reduce(input_v, pvector([]))[0])==138

Now try with the puzzle input:

In [74]:
puzzle_input=open('inputs/day8').read()

input_v=pvector([int(x) for x in puzzle_input.strip().split()])

In [75]:
sum(reduce(input_v, pvector([]))[0])

45865

The right answer!

## Part 2

I'm hoping that this part is easier to solve than it is to actually understand.

Probably easiest to alter `reduce` so that in the recursive case, we return a list of child nodes, and work from those.

In [90]:
def parse(input_v):
    '''
    parse the node at the front of the input vector. Return pair
    of node value and the rest of the input.
    '''
    if input_v[0]==0:
        # If no children, return sum of metadata
        return(sum(input_v[2:input_v[1]+2]),
               input_v[input_v[1]+2:])
    else:
        to_parse_v=input_v[2:]
        child_values_ls=[]
        for i in range(input_v[0]):
            (value, to_parse_v)=parse(to_parse_v)
            child_values_ls.append(value)
        metadata_values=to_parse_v[:input_v[1]]
        value_out=sum([child_values_ls[i-1] for i in metadata_values if i<=len(child_values_ls)])
        return (value_out, 
                to_parse_v[input_v[1]:])

In [91]:
test_input="2 3 0 3 10 11 12 1 1 0 1 99 2 1 1 2"

input_v=pvector([int(x) for x in test_input.strip().split()])
input_v

pvector([2, 3, 0, 3, 10, 11, 12, 1, 1, 0, 1, 99, 2, 1, 1, 2])

In [94]:
assert parse(input_v)[0]==66

Seems OK. Try with the puzzle input:

In [96]:
puzzle_input=open('inputs/day8').read()

input_v=pvector([int(x) for x in puzzle_input.strip().split()])

In [97]:
parse(input_v)[0]

22608